In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


## Self-Consistency Prompting in Language Models

Self-consistency prompting was introduced in the March 2022 paper ["Self-Consistency Improves Chain of Thought Reasoning in Language Models"](https://arxiv.org/pdf/2203.11171.pdf) by Xuezhi Wang, et. al.

# 🤔 **What is Self-Consistency Prompting?**

- Focuses on exploring different reasoning paths for complex problems.

- Aims for reliable answers by checking consistency across various thought processes.

### 🌟 **Differences from Other Prompting Techniques**

- Traditional CoT: Generates short sentences mimicking human reasoning for solving tasks.

- Self-Consistency: Samples multiple reasoning paths and finds the most consistent answer.

- It's unsupervised and works with pre-trained models, unlike methods needing extra training or human annotations.

### **🛠️ Constructing a Self-Consistency Prompt**


<figure>
  <img src="https://pbs.twimg.com/media/Ff73Y-RaEAANaqT.jpg" alt="Image Description" style="width:100%">
  <figcaption>The self-consistency method contains three steps: (1) prompt a language model using chain-of-thought (CoT) prompting; (2) replace the “greedy decode” in CoT prompting by sampling from the language model’s decoder to generate a diverse set of reasoning paths; and (3) marginalize out the reasoning paths and aggregate by choosing the most consistent answer in the final answer set.</figcaption>
  <a href="https://twitter.com/denny_zhou/status/1584978661668966401">Image Source</a>
</figure>


1. **Start with CoT**: Use chain-of-thought prompting as the base.

2. **Sample Diverse Paths**: Use "sample-and-marginalize" decoding to get various reasoning paths.

3. **Marginalize for Consistency**: Find the most consistent answer from these different paths.

### 🔍 **Examples in Action**

- **Chain-of-Thought**: For the question "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?", instead of directly answering "5", the model might respond with the reasoning: "There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5."

- **Self-Consistency**: For a question about how much Janet makes from selling eggs, the model might generate multiple reasoning paths like:
  1. "She has 16 - 3 - 4 = 9 eggs left. So she makes $2 * 9 = $18 per day."

  2. "This means she sells the remainder for $2 * (16 - 4 - 3) = $26 per day."
  
  3. "She eats 3 for breakfast, so she has 16 - 3 = 13 left. Then she bakes muffins, so she has 13 - 4 = 9 eggs left. So she has 9 eggs * $2 = $18."

  The model then aggregates these paths to determine the most consistent answer, which in this case is "$18 per day."


🚀 **Impact of Self-Consistency Prompting**
- Enhances model reasoning by considering multiple paths.
- Shown to boost performance in arithmetic and commonsense reasoning tasks.

# Begin by setting up CoT prompts

We're following the same pattern from the Chain of Thought lesson.

1. Downloading CoT prompt datasets from HuggingFace

2. Downloading embeddings model from HuggingFace

3. Creating prompt template for CoT

4. Creating an example selector

5. Construct the prompt

In [ ]:
pip install datasets

In [4]:
%%capture
from datasets import load_dataset

dataset = load_dataset("kaist-ai/CoT-Collection", split="train", trust_remote_code=True)

dataset = dataset.remove_columns(['task', 'type'])

dataset = dataset.shuffle(seed=42)

dataset = dataset.select(range(10_000))

dataset = dataset.to_pandas()

selected_examples = dataset.to_dict(orient='records')

In [ ]:
pip install langchain-community sentence-transformers

In [9]:
%%capture
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"

encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [10]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import FAISS
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

prefix = "Consider the following as examples of how to reason:"

examples_template = """Query: {source}

Rationale: {rationale}

Response: {target}
"""

suffix = """Using a similar reasoning approach, answer the users question which is delimited by triple backticks.

User question: ```{input}```

Take a deep breath, break down the user's query step-by-step, and provide a clear chain of thought in your response."
"""

examples_prompt = PromptTemplate(
    input_variables=["source", "rationale", "target"],
    template=examples_template
)

In [ ]:
pip install faiss-gpu

In [13]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    selected_examples,
    embeddings,
    FAISS,
    k=5,
)

mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"]
)

In [14]:
query = """There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
"""
prompt = mmr_prompt.format(input=query)

In [15]:
print(prompt)

Consider the following as examples of how to reason:

Query: Extract the answer to the question from the following context.
Question: What was once only found in large computers?
Context: Some computers are designed to distribute their work across several CPUs in a multiprocessing configuration, a technique once employed only in large and powerful machines such as supercomputers, mainframe computers and servers. Multiprocessor and multi-core (multiple CPUs on a single integrated circuit) personal and laptop computers are now widely available, and are being increasingly used in lower-end markets as a result.

Rationale: The context mentions that multiprocessing configurations were once only found in large computers. Therefore, the answer is "several CPUs in a multiprocessing configuration."

Response: several CPUs in a multiprocessing configuration


Query: Given a math word problem, answer the following question. You might need to apply addition or subtraction mathematical operators on

# Self-Consistency Prompt

In [16]:
sc_template = """Based on the responses (delimited by < >) to the following query, \
(delimited by triple backticks) return the response that occurs most frequently.

Query: ```{query}```

Responses: <{responses}>
"""

sc_prompt = PromptTemplate.from_template(sc_template)

### Generating multiple responses


Use the `n` parameter to generate alternative responses. Increase `n` to explore different variations.



In [ ]:
pip install langchain-openai

In [19]:
from langchain_openai import OpenAI

# we'll use the default model here, gpt-3.5-turbo-instruct
llm = OpenAI(n=5)

generations = llm.generate([prompt])

In [20]:
generations.generations

[[Generation(text="\nRationale: The user's query is asking about the current number of computers in the server room after a certain number of installations. First, we know that there were initially 9 computers in the server room. Then, 5 more computers were installed each day from Monday to Thursday. This means that 5 computers were installed on each of those 4 days, resulting in a total of 20 additional installations (5 x 4 = 20). Therefore, the current number of computers in the server room is 9 + 20 = 29.\n\nResponse: 29 computers are now in the server room.", generation_info={'finish_reason': 'stop', 'logprobs': None}),
  Generation(text='\nRationale: The user is asking for the total number of computers in the server room after five more were installed each day from Monday to Thursday. This can be solved by first finding how many computers were installed in total from Monday to Thursday, then adding that to the initial nine computers in the room.\n\nStep 1: Find the number of compu

In [21]:
responses = []
for item in generations.generations[0]:
    response_index = item.text.find("Response: ")
    if response_index != -1:
        response = item.text[response_index + len("Response: "):].strip()
        responses.append(response)

In [22]:
responses

['29 computers are now in the server room.',
 '59 computers',
 '29',
 '31 computers are now in the server room.',
 '29']

In [23]:
llm = OpenAI()

final_prompt = sc_prompt.format(query=query, responses=str(responses))

print(final_prompt)

Based on the responses (delimited by < >) to the following query, (delimited by triple backticks) return the response that occurs most frequently.

Query: ```There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
```

Responses: <['29 computers are now in the server room.', '59 computers', '29', '31 computers are now in the server room.', '29']>



In [24]:
print(llm.invoke(final_prompt))


29 computers are now in the server room.
